# Q2.1 Classic Machine Learning Methods

In [52]:
import pandas as pd
import numpy as np
from project_1.config import PROCESSED_DATA_DIR, PROJ_ROOT

SEED = 42

In [53]:
# Load the data from Parquet files
sets_dict = {}
sets = ["a", "b", "c"]

for set_name in sets:
    directory = PROCESSED_DATA_DIR / f"set_{set_name}_final.parquet"
    temp_set = pd.read_parquet(directory)
    sets_dict[f"set_{set_name}"] = temp_set

# Assure the loading was correct
print(sets_dict["set_a"].shape)
sets_dict["set_a"].head(10)


(183416, 43)


,RecordID,Time,Gender,Height,Weight,Age,Albumin,Cholesterol,DiasABP,HCO3,HCT,HR,Mg,MAP,Na,NIDiasABP,NIMAP,NISysABP,SysABP,PaCO2,PaO2,Platelets,RespRate,Temp,ALP,ALT,AST,BUN,Bilirubin,Creatinine,FiO2,GCS,Glucose,ICUType,K,Lactate,MechVent,Urine,WBC,pH,SaO2,TroponinT,TroponinI
0,132539.0,2025-03-10 00:00:00,0.0,-0.953317,0.283445,-0.596332,1.004449,-0.069047,-0.739159,-1.269243,0.906679,0.795048,-0.736292,0.063050,-0.175138,-2.816704,2.654924,-3.122410,-0.666106,0.611934,3.813686,0.270109,-0.951030,-1.950847,-0.039216,14.789474,3.263158,-0.20,1.666667,0.333333,5.0,0.166667,-0.708333,0.5,0.625,-0.583333,0.0,2.96,0.271429,-0.750,1.0,0.933333,-0.224490
1,132539.0,2025-03-10 01:00:00,0.0,-0.953317,0.283445,-0.596332,-0.232975,1.997003,0.232700,-2.924060,-1.196905,-0.683119,-1.720481,-0.380835,-1.815592,0.270733,0.883708,1.385468,1.882994,-0.711042,2.271499,0.798617,-0.066706,-1.892239,-0.490196,-0.333333,-0.390351,2.30,-0.444444,-0.666667,5.0,0.166667,0.354167,0.5,0.375,-0.166667,0.0,3.20,0.000000,0.750,-15.5,-0.177778,0.408163
2,132539.0,2025-03-10 02:00:00,0.0,-0.953317,0.283445,-0.596332,-1.132919,2.628972,2.440210,0.385573,2.256887,-1.535907,-1.966528,-0.303155,-0.878190,0.301301,0.602040,0.821984,-0.666106,-0.470501,-0.682470,-1.315414,-0.243571,-1.892239,0.862745,-0.245614,0.114035,0.95,0.111111,-0.666667,2.5,0.166667,-0.708333,0.5,0.625,2.000000,0.0,-0.40,2.185714,-0.250,1.0,0.333333,-0.285714
3,132539.0,2025-03-10 03:00:00,0.0,-0.953317,0.283445,-0.596332,-0.120482,-1.721888,1.169851,-0.441835,0.456610,-1.422202,0.247897,-0.510302,0.293563,-0.310072,-0.082054,0.296064,0.183594,1.333557,3.416799,0.798617,-0.066706,-1.892239,0.862745,40.929825,1.394737,2.25,0.111111,-0.166667,0.0,0.166667,0.250000,0.5,-1.000,-0.416667,0.0,0.72,2.185714,-0.250,0.5,0.333333,2.040816
4,132539.0,2025-03-10 04:00:00,0.0,-0.953317,0.283445,-0.596332,-0.120482,1.462261,-0.392067,-2.510356,0.378337,-0.398856,0.493944,-0.741492,-0.409489,-0.310072,-0.263191,-0.042027,-1.013711,-1.432666,0.842708,0.798617,0.110158,0.979508,-0.647059,40.929825,0.192982,2.25,26.666667,0.666667,-1.0,0.166667,0.854167,0.5,0.125,0.000000,0.0,-0.16,-0.957143,0.375,1.0,0.200000,0.795918
5,132539.0,2025-03-10 05:00:00,0.0,-0.953317,0.283445,-0.596332,1.341928,1.146276,-0.235875,0.178721,0.378337,-0.739971,0.493944,-0.311478,0.293563,-0.310072,-0.263191,-0.042027,2.076107,0.010581,-0.642781,0.788830,0.110158,0.979508,0.862745,-0.192982,2.271930,-0.20,26.666667,0.000000,0.0,0.166667,0.104167,0.5,-2.125,1.666667,0.0,-0.16,1.557143,-0.375,0.0,-0.244444,0.183673
6,132539.0,2025-03-10 06:00:00,0.0,-0.953317,0.283445,-0.596332,1.341928,-0.506564,-0.322648,-0.234983,0.378337,-0.796824,-0.490245,-1.102149,-0.409489,-0.738034,-0.625465,-0.192289,-1.206824,1.574098,-0.767517,0.103727,-0.420436,0.979508,-0.627451,-0.403509,17.210526,0.40,0.000000,0.666667,-1.0,0.166667,0.854167,0.5,0.625,0.833333,0.0,0.72,1.557143,-0.125,0.0,0.800000,-0.081633
7,132539.0,2025-03-10 08:00:00,0.0,-0.953317,0.283445,-0.596332,-1.357905,-0.117660,-0.010265,1.419833,0.378337,-1.308497,-1.720481,-0.491807,0.293563,-0.493484,-0.525236,-0.304986,1.226407,-0.350231,-0.535055,-0.356270,-0.774165,1.331151,-0.058824,0.000000,-0.491228,-0.40,2.555556,1.000000,-1.0,0.166667,0.250000,0.5,-0.250,0.250000,0.0,0.32,-0.657143,-0.625,0.0,20.911111,8.408163
8,132539.0,2025-03-10 09:00:00,0.0,-0.953317,0.283445,-0.596332,-1.132919,0.344163,0.232700,-0.028131,0.378337,-1.308497,-0.490245,-0.824721,0.527913,-0.065523,-0.344099,-0.492814,0.453953,-1.432666,-0.450008,-0.669460,-0.951030,1.331151,-0.607843,0.052632,0.596491,-0.15,4.777778,1.000000,3.0,0.166667,0.083333,0.5,-0.250,-0.416667,0.0,0.00,-0.657143,-0.250,0.0,-0.088889,-0.020408
9,132539.0,2025-03-10 10:00:00,0.0,-0.953317,0.283445,-0.596332,-1.470399,0.076791,0.302119,0.385573,0.378337,-1.194792,-1.720481,-0.103407,0.293563,-0.554622,-0.424403,-0.042027,-0.498741,-0.350231,-0.552064,-0.072442,-0.420436,1.331151,-0.352941,0.052632,0.596491,-0.15,-0.444444,-0.333333,1.0,0.1

ICU TYpe da cavare

In [54]:

# Define file names
file_names = ["Outcomes-a.txt", "Outcomes-b.txt", "Outcomes-c.txt"]

# Directory path
base_path = PROJ_ROOT / "data" / "data_1" / "predicting-mortality-of-icu-patients-the-physionet-computing-in-cardiology-challenge-2012-1.0.0"

# Read files into DataFrames containing all variables
outcomes_a, outcomes_b, outcomes_c = [pd.read_csv(base_path / name) for name in file_names]

# Extract only the "RecordID" and "In-hospital_death" column into separate DataFrames
death_a, death_b, death_c = [df[["RecordID", "In-hospital_death"]] for df in [outcomes_a, outcomes_b, outcomes_c]]
death_a.head()

#CHECK for missing values in the outcome data
print(death_a.isnull().sum())
print(death_b.isnull().sum())
print(death_c.isnull().sum())



RecordID             0
In-hospital_death    0
dtype: int64
RecordID             0
In-hospital_death    0
dtype: int64
RecordID             0
In-hospital_death    0
dtype: int64


In [55]:
# Check if each "In-hospital_death" column contains only 0 and 1
for name, df in zip(["a", "b", "c"], [death_a, death_b, death_c]):
    print(f"\nValue counts for death_{name}:\n")
    print(df["In-hospital_death"].value_counts())
    print("\nContains only 0 and 1:", df["In-hospital_death"].isin([0, 1]).all())



Value counts for death_a:

In-hospital_death
0    3446
1     554
Name: count, dtype: int64

Contains only 0 and 1: True

Value counts for death_b:

In-hospital_death
0    3432
1     568
Name: count, dtype: int64

Contains only 0 and 1: True

Value counts for death_c:

In-hospital_death
0    3415
1     585
Name: count, dtype: int64

Contains only 0 and 1: True


So 1 means he died. We are going to fit a classifier to predict death. 

### Initial setup for the classifier to work

In [56]:
# Define the sets you want to modify
sets = ["set_a", "set_b", "set_c"]

# Delete the column ICUType from each set
for set_name in sets:
    sets_dict[set_name].drop(columns=["ICUType"], inplace=True)

# Check if the column was deleted by printing all the columns of set_a
sets_dict["set_a"].columns

Index(['RecordID', 'Time', 'Gender', 'Height', 'Weight', 'Age', 'Albumin',
       'Cholesterol', 'DiasABP', 'HCO3', 'HCT', 'HR', 'Mg', 'MAP', 'Na',
       'NIDiasABP', 'NIMAP', 'NISysABP', 'SysABP', 'PaCO2', 'PaO2',
       'Platelets', 'RespRate', 'Temp', 'ALP', 'ALT', 'AST', 'BUN',
       'Bilirubin', 'Creatinine', 'FiO2', 'GCS', 'Glucose', 'K', 'Lactate',
       'MechVent', 'Urine', 'WBC', 'pH', 'SaO2', 'TroponinT', 'TroponinI'],
      dtype='object')

In [57]:
# Define training, validation and testing sets
train_set = sets_dict["set_a"]
val_set = sets_dict["set_b"]
test_set = sets_dict["set_c"]


In [58]:

#change class of RecordID to int32
train_set["RecordID"] = train_set["RecordID"].astype("int32")
val_set["RecordID"] = val_set["RecordID"].astype("int32")
test_set["RecordID"] = test_set["RecordID"].astype("int32")


In [59]:
# Merge the training, validation and testing sets with the corresponding death DataFrames
train_set = train_set.merge(death_a, on="RecordID")
val_set = val_set.merge(death_b, on="RecordID") 
test_set = test_set.merge(death_c, on="RecordID")

# Check if the merge was successful by printing the first 5 rows of the training set
train_set.head()

,RecordID,Time,Gender,Height,Weight,Age,Albumin,Cholesterol,DiasABP,HCO3,HCT,HR,Mg,MAP,Na,NIDiasABP,NIMAP,NISysABP,SysABP,PaCO2,PaO2,Platelets,RespRate,Temp,ALP,ALT,AST,BUN,Bilirubin,Creatinine,FiO2,GCS,Glucose,K,Lactate,MechVent,Urine,WBC,pH,SaO2,TroponinT,TroponinI,In-hospital_death
0,132539,2025-03-10 00:00:00,0.0,-0.953317,0.283445,-0.596332,1.004449,-0.069047,-0.739159,-1.269243,0.906679,0.795048,-0.736292,0.063050,-0.175138,-2.816704,2.654924,-3.122410,-0.666106,0.611934,3.813686,0.270109,-0.951030,-1.950847,-0.039216,14.789474,3.263158,-0.20,1.666667,0.333333,5.0,0.166667,-0.708333,0.625,-0.583333,0.0,2.96,0.271429,-0.750,1.0,0.933333,-0.224490,0
1,132539,2025-03-10 01:00:00,0.0,-0.953317,0.283445,-0.596332,-0.232975,1.997003,0.232700,-2.924060,-1.196905,-0.683119,-1.720481,-0.380835,-1.815592,0.270733,0.883708,1.385468,1.882994,-0.711042,2.271499,0.798617,-0.066706,-1.892239,-0.490196,-0.333333,-0.390351,2.30,-0.444444,-0.666667,5.0,0.166667,0.354167,0.375,-0.166667,0.0,3.20,0.000000,0.750,-15.5,-0.177778,0.408163,0
2,132539,2025-03-10 02:00:00,0.0,-0.953317,0.283445,-0.596332,-1.132919,2.628972,2.440210,0.385573,2.256887,-1.535907,-1.966528,-0.303155,-0.878190,0.301301,0.602040,0.821984,-0.666106,-0.470501,-0.682470,-1.315414,-0.243571,-1.892239,0.862745,-0.245614,0.114035,0.95,0.111111,-0.666667,2.5,0.166667,-0.708333,0.625,2.000000,0.0,-0.40,2.185714,-0.250,1.0,0.333333,-0.285714,0
3,132539,2025-03-10 03:00:00,0.0,-0.953317,0.283445,-0.596332,-0.120482,-1.721888,1.169851,-0.441835,0.456610,-1.422202,0.247897,-0.510302,0.293563,-0.310072,-0.082054,0.296064,0.183594,1.333557,3.416799,0.798617,-0.066706,-1.892239,0.862745,40.929825,1.394737,2.25,0.111111,-0.166667,0.0,0.166667,0.250000,-1.000,-0.416667,0.0,0.72,2.185714,-0.250,0.5,0.333333,2.040816,0
4,132539,2025-03-10 04:00:00,0.0,-0.953317,0.283445,-0.596332,-0.120482,1.462261,-0.392067,-2.510356,0.378337,-0.398856,0.493944,-0.741492,-0.409489,-0.310072,-0.263191,-0.042027,-1.013711,-1.432666,0.842708,0.798617,0.110158,0.979508,-0.647059,40.929825,0.192982,2.25,26.666667,0.666667,-1.0,0.166667,0.854167,0.125,0.000000,0.0,-0.16,-0.957143,0.375,1.0,0.200000,0.795918,0


In [60]:
# Define aggregation rules
aggregation_rules = {
    "Age": "last",
    "Gender": "last",
    "Height": "last",
    "Albumin": "last",
    "ALP": "last",
    "ALT": "last",
    "AST": "last",
    "Bilirubin": "last",
    "BUN": "last",
    "Cholesterol": "last",
    "Creatinine": "last",
    "DiasABP": "mean",
    "FiO2": "mean",
    "GCS": "min",
    "Glucose": "mean",
    "HCO3": "last",
    "HCT": "last",
    "HR": "mean",
    "K": "last",
    "Lactate": "max",
    "Mg": "last",
    "MAP": "mean",
    "MechVent": "last",
    "Na": "last",
    "NIDiasABP": "mean",
    "NIMAP": "mean",
    "NISysABP": "mean",
    "PaCO2": "last",
    "PaO2": "mean",
    "pH": "last",
    "Platelets": "last",
    "RespRate": "mean",
    "SaO2": "mean",
    "SysABP": "mean",
    "Temp": "max",
    "TroponinI": "max",
    "TroponinT": "max",
    "Urine": "sum",
    "WBC": "last",
    "Weight": "last",
    "In-hospital_death": "max"  # If any 1 exists for a patient, return 1
}

# Perform aggregation
train_aggregated = train_set.groupby("RecordID").agg(aggregation_rules).reset_index()
val_aggregated = val_set.groupby("RecordID").agg(aggregation_rules).reset_index()
test_aggregated = test_set.groupby("RecordID").agg(aggregation_rules).reset_index()


# Display the processed dataset
train_aggregated.head()

,RecordID,Age,Gender,Height,Albumin,ALP,ALT,AST,Bilirubin,BUN,Cholesterol,Creatinine,DiasABP,FiO2,GCS,Glucose,HCO3,HCT,HR,K,Lactate,Mg,MAP,MechVent,Na,NIDiasABP,NIMAP,NISysABP,PaCO2,PaO2,pH,Platelets,RespRate,SaO2,SysABP,Temp,TroponinI,TroponinT,Urine,WBC,Weight,In-hospital_death
0,132539,-0.596332,0.0,-0.953317,-0.120482,0.372549,-0.263158,0.289474,0.222222,-0.55,-1.721888,-0.333333,0.203848,0.052083,0.000000,0.700521,1.006129,-0.286982,-0.796824,-0.125,4.000000,-0.244197,0.187026,0.0,-0.643839,-0.527237,-0.332557,-0.146115,-0.951583,0.052479,-1.375,-0.209463,-0.328319,-0.536458,0.158650,1.448365,8.408163,20.911111,38.320000,-0.300000,0.283445,0
1,132540,0.667051,1.0,0.615786,-0.120482,0.372549,-0.263158,0.289474,0.222222,0.10,-1.721888,0.666667,-0.177908,-0.663265,-1.833333,-0.163690,0.178721,-0.463096,-0.442946,-0.750,6.750000,-0.244197,-0.249406,0.0,-0.878190,-0.007354,-0.186115,-0.159643,0.732204,-0.219050,-0.250,-0.698822,-0.011361,-0.428571,-0.151136,1.096722,8.408163,20.911111,30.265333,0.257143,-0.232912,0
2,132541,-1.170597,0.0,-0.432085,-1.020426,0.431373,0.771930,0.771930,2.333333,-0.80,-1.721888,-1.000000,0.260251,0.052083,-1.500000,0.001302,0.385573,-0.463096,-0.173222,-0.500,0.083333,-0.736292,0.162366,0.0,-0.175138,1.348066,1.340397,0.672112,-0.350231,-0.117143,1.000,-0.914140,0.004531,-0.885417,0.175950,2.386078,8.408163,20.911111,24.688000,-0.757143,-1.078178,0
3,132543,0.207639,1.0,1.156482,1.341928,0.431373,-0.333333,-0.535088,-0.555556,-0.45,-1.721888,-0.333333,0.191687,0.030612,0.000000,-0.330782,1.006129,0.887111,-0.968058,-0.375,6.750000,-0.244197,0.164868,0.0,-0.409489,0.326567,0.374225,0.128808,-0.951583,0.038753,-1.375,0.759468,-0.746235,-0.525510,0.173347,0.041795,8.408163,20.911111,141.272000,-0.514286,0.143735,0
4,132545,1.356169,0.0,-1.117689,0.104504,0.372549,-0.263158,0.289474,0.222222,0.30,-1.721888,0.166667,0.142973,-0.127660,0.166667,-0.452571,-0.648687,-0.169573,-0.687352,0.000,6.750000,-0.982339,0.186437,0.0,0.059212,-0.641125,0.000569,0.610444,-0.951583,0.006593,-1.375,-1.070735,-0.051654,-0.239362,0.154832,0.979508,8.408163,20.911111,-2.032000,-0.957143,-0.835986,0


# Model 1 - Logistic Regression

In [61]:
# Separate Predictors (X) and Target (y)
X_train = train_aggregated.drop(columns=["RecordID", "In-hospital_death"])
y_train = train_aggregated["In-hospital_death"]

X_val = val_aggregated.drop(columns=["RecordID", "In-hospital_death"])
y_val = val_aggregated["In-hospital_death"]

X_test = test_aggregated.drop(columns=["RecordID", "In-hospital_death"])
y_test = test_aggregated["In-hospital_death"]

# Visualize the shape of the datasets
print(X_train.shape, y_train.shape)


(4000, 40) (4000,)


In [62]:
#El bondiano fa una logistica daje
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score

# --- Assume these DataFrames are already defined ---
# train_df = pd.read_csv("train.csv")
# valid_df = pd.read_csv("validate.csv")
# test_df  = pd.read_csv("test.csv")

# Separate features and target for each set.
# Adjust the column name "target" to your actual target column name.

# Create and train the Logistic Regression classifier.
clf = LogisticRegression(random_state=SEED, max_iter=1000)
clf.fit(X_train, y_train)

# Optionally, evaluate on the validation set.
y_valid_proba = clf.predict_proba(X_val)[:, 1]  # probability for the positive class
roc_auc_valid = roc_auc_score(y_val, y_valid_proba)
auprc_valid = average_precision_score(y_val, y_valid_proba)
print(f"Validation ROC AUC: {roc_auc_valid:.3f}, AUPRC: {auprc_valid:.3f}")

# Evaluate on the test set.
y_test_proba = clf.predict_proba(X_test)[:, 1]
roc_auc_test = roc_auc_score(y_test, y_test_proba)
auprc_test = average_precision_score(y_test, y_test_proba)
print(f"Test ROC AUC: {roc_auc_test:.3f}, AUPRC: {auprc_test:.3f}")

Validation ROC AUC: 0.780, AUPRC: 0.397
Test ROC AUC: 0.769, AUPRC: 0.386


# Model 2 - Random Forest

In [63]:
# Use Random Forest to predict the target
from sklearn.ensemble import RandomForestClassifier

# Create and train the Random Forest classifier.
clf = RandomForestClassifier(random_state=SEED)
clf.fit(X_train, y_train)

# Optionally, evaluate on the validation set.
y_valid_proba = clf.predict_proba(X_val)[:, 1]  # probability for the positive class
roc_auc_valid = roc_auc_score(y_val, y_valid_proba)
auprc_valid = average_precision_score(y_val, y_valid_proba)
print(f"Validation ROC AUC: {roc_auc_valid:.3f}, AUPRC: {auprc_valid:.3f}")

# Evaluate on the test set.
y_test_proba = clf.predict_proba(X_test)[:, 1]
roc_auc_test = roc_auc_score(y_test, y_test_proba)
auprc_test = average_precision_score(y_test, y_test_proba)
print(f"Test ROC AUC: {roc_auc_test:.3f}, AUPRC: {auprc_test:.3f}")

Validation ROC AUC: 0.779, AUPRC: 0.421
Test ROC AUC: 0.790, AUPRC: 0.424


## A little comment on the results:

AUC Score is pretty good. However, AUPRC is not that high. A problem that could arise is class imbalance. In fact, we have two classes, where the size of the 0 class is 6x the size of the 1 class. However, all three sets share the same problem, so it should not be a big problem. Another possible implementation could be using a dimensionality reduction technique, or some feature engineering.

# Q2.1 - Feature Engineering

We could use solutions such as *_Feature Lagging_*, *_Temporal Differences_* or *_Rolling Statistics_* to better capture the time-series trends.

### Attempt 1 - Feature Lagging

In [64]:
"""QUESTO è QUELLO VECCHIO:
from tsfresh import extract_relevant_features
from tsfresh.feature_extraction.settings import MinimalFCParameters

# Define the column names
id_column = "RecordID"  
time_column = "Time"
target_column = "In-hospital_death"  


# Define features and target
X = train_set.drop(columns=[target_column])  # Keep all features, including Time
y = train_set.groupby(id_column)[target_column].max()  # Ensure one label per RecordID

# Extract relevant features
X_train_relevant = extract_relevant_features(X, 
                                       y=y, 
                                       column_id=id_column, 
                                       column_sort=time_column,
                                       default_fc_parameters=MinimalFCParameters()
                                       ) # Use Time as the sorting column

# Display extracted features
X_train_relevant.head()
"""

'QUESTO è QUELLO VECCHIO:\nfrom tsfresh import extract_relevant_features\nfrom tsfresh.feature_extraction.settings import MinimalFCParameters\n\n# Define the column names\nid_column = "RecordID"  \ntime_column = "Time"\ntarget_column = "In-hospital_death"  \n\n\n# Define features and target\nX = train_set.drop(columns=[target_column])  # Keep all features, including Time\ny = train_set.groupby(id_column)[target_column].max()  # Ensure one label per RecordID\n\n# Extract relevant features\nX_train_relevant = extract_relevant_features(X, \n                                       y=y, \n                                       column_id=id_column, \n                                       column_sort=time_column,\n                                       default_fc_parameters=MinimalFCParameters()\n                                       ) # Use Time as the sorting column\n\n# Display extracted features\nX_train_relevant.head()\n'

In [65]:
from tsfresh import extract_features, select_features
from tsfresh.feature_extraction.settings import MinimalFCParameters
from tsfresh.utilities.dataframe_functions import impute

# Define the column names
id_column = "RecordID"
time_column = "Time"
target_column = "In-hospital_death"

# Step 1: Extract all features from train_set using MinimalFCParameters
X_train_extracted = extract_features(
    train_set.drop(columns=[target_column]),
    column_id=id_column,
    column_sort=time_column,
    default_fc_parameters=MinimalFCParameters()  # Use minimal features
)

# Impute missing values (tsfresh requires no NaNs)
X_train_extracted = impute(X_train_extracted)

# Step 2: Select relevant features based on the target variable
y_train = train_set.groupby(id_column)[target_column].max()
X_train_relevant = select_features(X_train_extracted, y_train)

# Step 3: Store the relevant feature names
relevant_feature_names = X_train_relevant.columns

# Step 4: Extract the same features for val_set
X_val_extracted = extract_features(
    val_set.drop(columns=[target_column]),
    column_id=id_column,
    column_sort=time_column,
    default_fc_parameters=MinimalFCParameters()  # Use the same settings
)
X_val_extracted = impute(X_val_extracted)  # Impute missing values
X_val_relevant = X_val_extracted[relevant_feature_names]  # Keep only relevant features

# Step 5: Extract the same features for test_set
X_test_extracted = extract_features(
    test_set.drop(columns=[target_column]),
    column_id=id_column,
    column_sort=time_column,
    default_fc_parameters=MinimalFCParameters()  # Use the same settings
)
X_test_extracted = impute(X_test_extracted)  # Impute missing values
X_test_relevant = X_test_extracted[relevant_feature_names]  # Keep only relevant features

# Now X_train_relevant, X_val_relevant, and X_test_relevant have the same features
print("Train set relevant features:", X_train_relevant.shape)
print("Validation set relevant features:", X_val_relevant.shape)
print("Test set relevant features:", X_test_relevant.shape)

Feature Extraction: 100%|██████████| 20/20 [00:25<00:00,  1.29s/it]


Train set relevant features: (4000, 215)
Validation set relevant features: (4000, 215)
Test set relevant features: (4000, 215)


In [66]:
# For train_set
y_train = train_set.groupby(id_column)[target_column].max()

# For val_set
y_val = val_set.groupby(id_column)[target_column].max()

# For test_set
y_test = test_set.groupby(id_column)[target_column].max()


In [67]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.preprocessing import StandardScaler

# --- Scale the features ---
scaler = StandardScaler()

# Fit the scaler to the training data and transform
X_train_scaled = scaler.fit_transform(X_train_relevant)

# Transform the validation and test sets using the same scaler
X_val_scaled = scaler.transform(X_val_relevant)
X_test_scaled = scaler.transform(X_test_relevant)

# --- Create and train the Logistic Regression classifier ---
clf = LogisticRegression(random_state=42, max_iter=5000)  # Increase iterations to 5000
clf.fit(X_train_scaled, y_train)

# --- Evaluate on the validation set ---
y_val_proba = clf.predict_proba(X_val_scaled)[:, 1]  # Probability for the positive class
roc_auc_val = roc_auc_score(y_val, y_val_proba)
auprc_val = average_precision_score(y_val, y_val_proba)
print(f"Validation ROC AUC: {roc_auc_val:.3f}, AUPRC: {auprc_val:.3f}")

# --- Evaluate on the test set ---
y_test_proba = clf.predict_proba(X_test_scaled)[:, 1]
roc_auc_test = roc_auc_score(y_test, y_test_proba)
auprc_test = average_precision_score(y_test, y_test_proba)
print(f"Test ROC AUC: {roc_auc_test:.3f}, AUPRC: {auprc_test:.3f}")


Validation ROC AUC: 0.821, AUPRC: 0.438
Test ROC AUC: 0.817, AUPRC: 0.469


Since there are 215 features, we'll try with some regularization (l2)

In [72]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

# --- Scale the features ---
scaler = StandardScaler()

# Fit the scaler to the training data and transform
X_train_scaled = scaler.fit_transform(X_train_relevant)

# Transform the validation and test sets using the same scaler
X_val_scaled = scaler.transform(X_val_relevant)
X_test_scaled = scaler.transform(X_test_relevant)

# --- Define the Logistic Regression model ---
# Use L2 regularization (default) and set up the solver
log_reg = LogisticRegression(random_state=42, max_iter=5000, penalty='l2', solver='liblinear')

# --- Set up the grid search for hyperparameter tuning ---
# Define the range of C values to test
param_grid = {'C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}

# Use ROC AUC as the scoring metric for grid search
grid_search = GridSearchCV(estimator=log_reg, param_grid=param_grid, scoring='roc_auc', cv=5, verbose=1)

# --- Perform grid search on the training data ---
grid_search.fit(X_train_scaled, y_train)

# --- Get the best model and its parameters ---
best_clf = grid_search.best_estimator_
best_C = grid_search.best_params_['C']
print(f"Best regularization parameter (C): {best_C}")

# --- Evaluate on the validation set using the best model ---
y_val_proba = best_clf.predict_proba(X_val_scaled)[:, 1]  # Probability for the positive class
roc_auc_val = roc_auc_score(y_val, y_val_proba)
auprc_val = average_precision_score(y_val, y_val_proba)
print(f"Validation ROC AUC: {roc_auc_val:.3f}, AUPRC: {auprc_val:.3f}")

# --- Evaluate on the test set using the best model ---
y_test_proba = best_clf.predict_proba(X_test_scaled)[:, 1]
roc_auc_test = roc_auc_score(y_test, y_test_proba)
auprc_test = average_precision_score(y_test, y_test_proba)
print(f"Test ROC AUC: {roc_auc_test:.3f}, AUPRC: {auprc_test:.3f}")

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Best regularization parameter (C): 0.01
Validation ROC AUC: 0.829, AUPRC: 0.458
Test ROC AUC: 0.827, AUPRC: 0.487
